## Description

I wants understand why my train is different on other people where i wrong?
I train default how i train another model detection and convolution but i see in kernels in kaggle people use another techicks to train nlp model and i can't understand why and i want test it.

Links to use:
- [22nd solution](https://www.kaggle.com/c/commonlitreadabilityprize/discussion/257302)

- [gtihub link](https://github.com/kurupical/commonlit)

In [ ]:
from IPython.display import clear_output
!pip install transformers
clear_output()

In [ ]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import  DataLoader, Dataset
import transformers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

df = pd.read_csv(path_tr)

SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def clean_text(txt):
    return re.sub('[^A-Za-z]+', ' ',str(txt).lower())
df['txt'] = df['excerpt'].apply(lambda x: clean_text(x))

In [ ]:
def make_folds(data: pd.DataFrame, split: int = 5):
    data['kfold'] = -1
    data = data.sample(frac =1).reset_index(drop=True)
    num_bins = int(np.floor(1 + np.log2(len(data))))
    data.loc[:, 'bins'] = pd.cut(
        data['target'], bins = num_bins, labels = False
    )
    kf = model_selection.StratifiedKFold(n_splits=split)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    data = data.drop("bins", axis=1)
    return data

In [ ]:
df_folds = make_folds(df, 5)
df_folds.kfold.value_counts()

3    567
1    567
2    567
0    567
4    566
Name: kfold, dtype: int64

In [ ]:
class CL_Dataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        token,
        max_len: int = 256,
        test: bool = False
        ) -> dict:
        self.data = data 
        self.max_len = max_len
        self.test = test
        self.token = token

    def __len__(self):
        return self.data.shape[0]

    def  __getitem__(self, idx: int):
        text = self.data.txt.iloc[idx]
        encode = self.token(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            add_special_tokens=True,            
            return_attention_mask=True,
            return_token_type_ids=False  
            )
        if self.test:
            target = 0
        else:
            target = self.data.target.iloc[idx]

        ids = encode["input_ids"]
        mask = encode["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(target, dtype = torch.float)  
        }

In [ ]:
def loss_fn(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))


def viz_curve(data: dict, title: str) -> None:
    epochs = list(range(1, EPOCH + 1))
    fig = make_subplots(
        rows=2, cols=2,
        specs=[
               [{"colspan": 2}, None],
               [{}, {}],
               ],
               vertical_spacing=0.09,
               subplot_titles=('Loss Curve',  'RMSE', 'LR')
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['train_loss'],
            mode='lines+markers',
            name='Train Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_loss'],
            mode='lines+markers',
            name='Valid Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_rms'],
            mode='lines+markers',
            name='RMSE'),     
            row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['lr'],
            mode='lines+markers',
            name='LR'), 
            row=2, col=2
    )
    fig.update_layout(
        height=600,
        width=600,
        showlegend=False,
        title = title,
        margin=dict(l=10, r=10, t=30, b=20),                     
        template="plotly_dark"    
    )
    fig.show()

def update_result(
    data: dict,
    predict: torch.tensor,
    target:torch.tensor
) -> None:
    loss = loss_fn(predict.squeeze(-1), target)
    data['losses'].append(loss.detach().cpu().numpy())
    data['all_pred'].append(predict.squeeze(-1).detach().cpu().numpy())
    data['all_target'].append(target.detach().cpu().numpy())
    return loss


def train(
    model:nn.Module, 
    loader: DataLoader,
    optimizer: transformers.AdamW,
    schedule: transformers.get_linear_schedule_with_warmup,
    batch: int,
    max_lenght: int
) -> list:
    model.train()
    torch.backends.cudnn.benchmark = True
    results = defaultdict(list)
    for input in loader:
        optimizer.zero_grad()
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'}   
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        loss = update_result(results, out, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()    
        schedule.step()
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp))  
    return losses, rmse


def validate(
    model:nn.Module, 
    loader: DataLoader, 
    batch: int,
    max_lenght: int
) -> list:
    model.eval()
    results = defaultdict(list)
    for input in loader:  
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'} 
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        _ = update_result(results, out, target)
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp)) 
    return losses, rmse


def showtime(model:nn.Module, data: pd.DataFrame, tokenizer:transformers.AutoTokenizer, fold: int) -> dict:    
    history = defaultdict(list)
    model = model.to(device)
    ttr = data[data.kfold != fold].reset_index(drop=True)
    vvl = data[data.kfold == fold].reset_index(drop=True)
    print(f'Fold: {fold + 1}, --- {ttr.shape, vvl.shape}')

    tr = CL_Dataset(ttr,tokenizer, MAX_LEN)
    vl = CL_Dataset(vvl,tokenizer, MAX_LEN)
    tr_loader = DataLoader(
        tr,
        batch_size=BATCH,
        shuffle=True
    )
    vl_loader = DataLoader(
        vl,
        batch_size=BATCH,
        shuffle=False
    )
    optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=True)
    steps = len(tr_loader) * EPOCH
    # steps = len(tr_loader)/BATCH * EPOCH
    lin_schedule = transformers.get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=1,
        num_training_steps=steps
    )
    best_rmse = np.inf
    for epoch in tqdm(range(EPOCH)):
        tr_loss, tr_rmse = train(model, tr_loader, optimizer, lin_schedule, BATCH, MAX_LEN)
        vl_loss, vl_rmse = validate(model, vl_loader, BATCH, MAX_LEN)
        lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(tr_loss)
        history['valid_loss'].append(vl_loss)
        history['valid_rms'].append(vl_rmse)
        history['lr'].append(lr)
        print(f'Epoch: {epoch}, lr: {lr}, train rmse: {tr_rmse}, vl rmse: {vl_rmse}, vl loss: {vl_loss}')
        if vl_rmse < best_rmse:
            print(f'Save rmse: {vl_rmse}')
            torch.save(model.state_dict(), f'{MODEL}_model_{fold}.pth')
            best_rmse = vl_rmse
    return history

In [ ]:
EPOCH = 2
BATCH = 8
MAX_LEN = 314

class Auto_NLP_model:
    def __init__(self, name: str, n_labels: int = 1):
        print(f'Auto NLP init model : {name}')
        self.name = name
        self.n_labels=  n_labels        
    
    def loads(self):
        print('Loading configuraiton...')
        model_config = transformers.AutoConfig.from_pretrained(
                pretrained_model_name_or_path=self.name,
                num_labels=self.n_labels)
        print('Loading tokenizer...')
        tokenizer = transformers.AutoTokenizer.from_pretrained(
                pretrained_model_name_or_path=self.name)
        print('Loading model...')
        model = transformers.AutoModelForSequenceClassification.from_pretrained(
                pretrained_model_name_or_path=self.name,
                config=model_config)
        return model, tokenizer

In [ ]:
"""
https://huggingface.co/transformers/pretrained_models.html
https://huggingface.co/transformers/model_doc/auto.html

"""
m = {
     'albert': 'albert-base-v2',
     'bert': 'bert-base-cased',
     'xlnet': 'xlnet-base-cased',
     'roberta': 'distilroberta-base',
     'xlm-roberta':'xlm-roberta-base',
     'deberta': 'microsoft/deberta-base',
     'xlm':'xlm-mlm-ende-1024',
     'camembert': 'camembert-base'
     }
m2 = {
    'xlm-roberta':'xlm-roberta-base',
    'deberta': 'microsoft/deberta-base',
    'xlm':'xlm-mlm-ende-1024',
    'camembert': 'camembert-base',
    'mbart': 'facebook/mbart-large-cc25', #memory batch 4 not ends error
    'funnel': 'funnel-transformer/small'
}

In [ ]:
EPOCH = 1
BATCH = 6
MAX_LEN = 314

for k, v in m2.items():
    print(k)
    model, tokenizer = Auto_NLP_model(v).loads()
    MODEL = model.__class__.__name__
    fold = 0
    title = f'Model: {k} Fold: {fold + 1}'
    history = showtime(model, df_folds, tokenizer, fold)
    viz_curve(history, title)

funnel
Auto NLP init model : funnel-transformer/small
Loading configuraiton...



Loading tokenizer...



Loading model...


Some weights of the model checkpoint at funnel-transformer/small were not used when initializing FunnelForSequenceClassification: ['decoder.layers.0.ffn.linear_2.bias', 'decoder.layers.1.ffn.linear_1.bias', 'decoder.layers.0.ffn.linear_1.weight', 'decoder.layers.1.attention.r_kernel', 'decoder.layers.0.attention.post_proj.weight', 'decoder.layers.1.attention.layer_norm.weight', 'decoder.layers.1.ffn.linear_2.bias', 'decoder.layers.1.attention.k_head.weight', 'decoder.layers.0.attention.k_head.weight', 'decoder.layers.1.attention.r_r_bias', 'decoder.layers.0.attention.seg_embed', 'decoder.layers.1.attention.post_proj.bias', 'decoder.layers.0.ffn.layer_norm.bias', 'decoder.layers.0.ffn.linear_1.bias', 'decoder.layers.0.attention.layer_norm.weight', 'decoder.layers.0.attention.r_r_bias', 'decoder.layers.0.attention.r_w_bias', 'decoder.layers.1.attention.layer_norm.bias', 'decoder.layers.1.attention.post_proj.weight', 'decoder.layers.0.attention.k_head.bias', 'decoder.layers.0.ffn.layer_no

Fold: 1, --- ((2267, 8), (567, 8))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



Epoch: 0, lr: 0.0, train rmse: 0.7323076725006104, vl rmse: 0.6071954965591431, vl loss: 0.5790224075317383
Save rmse: 0.6071954965591431


100%|██████████| 1/1 [02:06<00:00, 126.72s/it]


#save to load local

In [ ]:
for name in [ 'albert-large-v2', 'roberta-large',  'funnel-transformer/medium-base']:
    model_config = transformers.AutoConfig.from_pretrained(
                    pretrained_model_name_or_path=name,
                    num_labels=1)
    tokenizer = transformers.AutoTokenizer.from_pretrained(
                    pretrained_model_name_or_path=name)
    model = transformers.AutoModelForSequenceClassification.from_pretrained(
                    pretrained_model_name_or_path=name,
                    config=model_config)
    tokenizer.save_pretrained(f"/content/drive/MyDrive/CommonLit/model/model_{name}")
    model.save_pretrained(f"/content/drive/MyDrive/CommonLit/model/model_{name}")

Some weights of the model checkpoint at albert-large-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You s

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

Some weights of FunnelForSequenceClassification were not initialized from the model checkpoint at funnel-transformer/medium-base and are newly initialized: ['classifier.linear_hidden.bias', 'classifier.linear_out.bias', 'classifier.linear_hidden.weight', 'classifier.linear_out.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
